In [ ]:
!pip install tensorflow-hub
!pip install tensorflow-datasets

In [ ]:
import os
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

In [ ]:
print("Version : ", tf.__version__)
print("Egar Mode : ", tf.executing_eagerly())
print("Hub : ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices("GPU") else "not available")

In [ ]:
train_data, validation_data, test_data = tfds.load(name="imdb_reviews", split=('train[60%:]', 'train[:60%]', 'test'), as_supervised=True)

In [ ]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))

In [ ]:
train_examples_batch

In [ ]:
train_labels_batch

In [ ]:
embedding = "https://tfhub.dev/google/nnlm-en-dim128/2"
hub_layer = hub.KerasLayer(embedding, input_shape=[], dtype = tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

In [ ]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))
model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss = tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_data.shuffle(1000).batch(512), 
                    epochs = 10,
                    validation_data=validation_data.batch(512),
                    verbose = 1)

In [ ]:
result = model.evaluate(test_data.batch(512), verbose=2)

In [ ]:
for name, value in zip(model.metrics_names, result):
  print("%s : %3.f"%(name, value))